In [1]:
import os
os.environ["TRANSFORMERS_OFFLINE"] ="1"
os.environ["TRANSFORMERS_CACHE"] = "/home/gridsan/groups/maia_mechint/models"

In [2]:
cd ../

/home/gridsan/cguo/rep_e_icl/rep_e_icl


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import numpy as np
from repe import repe_pipeline_registry
repe_pipeline_registry()
from tasks import get_task_dataset
from modules import *

In [5]:
from modules import *

In [6]:
model_name_or_path = "meta-llama/Llama-2-7b-hf"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="auto", token=True).eval()
use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=use_fast_tokenizer, padding_side="left", legacy=False, token=True)
tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id
tokenizer.bos_token_id = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
rep_token = -1
hidden_layers = list(range(-1, -model.config.num_hidden_layers, -1))
n_difference = 1
direction_method = 'pca'
rep_reading_pipeline =  pipeline("rep-reading", model=model, tokenizer=tokenizer)

In [20]:
dataset_name = 'ade_corpus_v2-classification'
positive_prompt = "Pay attention to the following examples."
negative_prompt = "Don't pay attention to the following exmaples."
ntrain = 64 
test_num_examples = 3

dataset = get_task_dataset(dataset_name, positive_prompt, negative_prompt, ntrain=ntrain, test_num_examples=test_num_examples)

getting dataset for ade_corpus_v2-classification
['Not Related', 'Related']
data len: 32
['Not Related', 'Related']
data len: 32
['Not Related', 'Related']
data len: 1568


In [9]:
(dataset["test"]["data"][0], dataset["test"]["labels"][0])

('Pemphigus vulgaris (PV) is a rare but potentially fatal autoimmune bullous disorder which frequently affects the oral mucosa.\nNot Related\nThe clinical course suggests that caffeine, which is present in oolong tea, was mainly responsible for the rhabdomyolysis as well as the delirium, although severe hyponatremia has been reported to cause rhabdomyolysis on rare occasions.\nRelated\nTardive oculogyric crisis (OGC) is a dystonic syndrome that starts after long-term use of dopamine receptor antagonists.\n',
 'Not Related')

In [31]:
rep_reading_pipeline =  pipeline("rep-reading", model=model, tokenizer=tokenizer)
rep_reader = get_rep_reader(model, rep_reading_pipeline, dataset)

["[INST] Don't pay attention to the following exmaples. [/INST] A 57-year-old white male with unresectable pancreatic cancer received capecitabine in combination with radiation therapy followed by capecitabine alone given over approximately a 3-month time period.\nNot Related\nA 70-year-old man was admitted to our hospital because of dyspnea after taking an antihistaminic agent (homochlorcyclizine hydrochloride) for itching.\nRelated\n", '[INST] Pay attention to the following examples. [/INST] The ECG monitoring revealed sinus pause with markedly prolonged PQ interval and QRS width.\nNot Related\nThis observation illustrates that the pharmacological activation of the parathyroid CaR and other putative CaR on bone cells by calcimimetics did not protect against the occurrence of hypercalcemia of immobilization favored by a severe HPTH-II in a hemodialysis patient.\nNot Related\n', "[INST] Don't pay attention to the following exmaples. [/INST] Antiarrhythmic agents can cause pneumonitis.\

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 31.74 GiB total capacity; 27.33 GiB already allocated; 172.88 MiB free; 30.70 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [17]:
hidden_layers = get_hidden_layers(model)
H_tests = get_h_test(
    model, 
    rep_reading_pipeline, 
    rep_reader, 
    dataset
)
plot_correlation(rep_reader, H_tests, hidden_layers)

AssertionError: 

In [22]:
data = dataset["test"]["data"][1]
test_data = get_test_data(model, tokenizer, [data])
wanted_layers = range(-1, -32, -1)
rep_reader_scores_dict, rep_reader_scores_mean_dict = get_rep_reader_scores_dict(model, tokenizer, 
                                                                                 rep_reading_pipeline, 
                                                                                 rep_reader, 
                                                                                 test_data
                                                                                 )

KeyboardInterrupt: 

In [14]:
THRESHOLD = 0.0 # change this to alter sensitivity
input_ids = tokenizer.tokenize(test_data[0])
plot_lat_scans(input_ids, rep_reader_scores_dict, layer_slice=slice(20, -20))

ValueError: '▁I' is not in list

In [23]:
rep_control = get_rep_control(model, tokenizer)

In [26]:
max_layer = -5
min_layer = -18

layer_id = list(range(max_layer, min_layer, -1))
baseline, pos_results, neg_results = get_rep_controlled_results(rep_reader, rep_control, dataset["test"]["data"], layer_id)

Done with baseline results!
Done with control outputs!
Done with neg control outputs


In [27]:
dataset["test"]["labels"][0]

'Not Related'

In [28]:
acc_dict = get_acc_dict(baseline, pos_results, neg_results, dataset["test"]["labels"])
acc_dict

{'baseline_acc': 0.598, 'pos_acc': 0.511, 'neg_acc': 0.516}

In [29]:
for a, b, c in zip(baseline, pos_results, neg_results): 
    print(a, b, c)
    print("-----")

Not Related Not Related Not Related
-----
Not Related Not Related Not Related
-----
Case report: The patient was A 5
-----
Not Related Not Related Not Related
-----
Not Related Not Related Not Related
-----
Not Related Not Related Not Related
-----
The patient was The patient was The patient was
-----
Not Related The authors report A case of
-----
RESULTS The patient was RESULTS
-----
Related Related Related
-----
Not Related Not Related Not Related
-----
A 6 The use of A 6
-----
Not Related Not Related Not Related
-----
Not Related Not Related Not Related
-----
Not Related The patient had This is an
-----
Related Related Related
-----
Not Related Not Related Not Related
-----
Not Related Not Related Not Related
-----
Not Related Not Related Not Related
-----
Not Related Not Related Not Related
-----
Not Related Not Related Not Related
-----
Related Related Related
-----
OBJECTIVE CONCLUS Methyl
-----
Related The patient had Rheum
-----
Not Related Not Related Not Related
-----
Not Rel

In [40]:
acc_dict

{'baseline_acc': 0.9605633802816902,
 'pos_acc': 0.8366197183098592,
 'neg_acc': 0.7154929577464789}

'positive\n'